In [1]:
import pandas as pd
import numpy as np
import scipy as sp 
import os
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
from acquire import get_diabetic_data, get_id_data


pd.set_option('display.max_columns', 80)

In [2]:
df= get_diabetic_data()

In [16]:
df[['admission_type_id']]=df[['admission_type_id']].astype(str)

In [4]:
df2= get_id_data()

In [5]:
df2.

,description
admission_type_id,
1,Emergency
2,Urgent
3,Elective
4,Newborn
5,Not Available
...,...
22,Transfer from hospital inpt/same fac reslt in...
23,Born inside this hospital
24,Born outside this hospital


In [19]:
# reading csv files
data1 = df
data2 = df2
  
# using merge function by setting how='left'
output2 = pd.merge(data1, data2, 
                   on='admission_type_id', 
                   how='outer')
  
# displaying result
print(output2)
df= pd.DataFrame(output2)
df.head()

        patient_nbr       race  gender     age weight    admission_type_id  \
0         8222157.0  Caucasian  Female  [0-10)      ?                    6   
1         8222157.0  Caucasian  Female  [0-10)      ?                    6   
2         8222157.0  Caucasian  Female  [0-10)      ?                    6   
3         8222157.0  Caucasian  Female  [0-10)      ?                    6   
4         8222157.0  Caucasian  Female  [0-10)      ?                    6   
...             ...        ...     ...     ...    ...                  ...   
915968          NaN        NaN     NaN     NaN    NaN                   30   
915969          NaN        NaN     NaN     NaN    NaN                   27   
915970          NaN        NaN     NaN     NaN    NaN                   28   
915971          NaN        NaN     NaN     NaN    NaN                   29   
915972          NaN        NaN     NaN     NaN    NaN  admission_source_id   

        discharge_disposition_id  admission_source_id  time_in_

,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,description_x,description_y
0,8222157.0,Caucasian,Female,[0-10),?,6,25.0,1.0,1.0,?,Pediatrics-Endocrinology,41.0,0.0,1.0,0.0,0.0,0.0,250.83,?,?,1.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,NaN,NaN
1,8222157.0,Caucasian,Female,[0-10),?,6,25.0,1.0,1.0,?,Pediatrics-Endocrinology,41.0,0.0,1.0,0.0,0.0,0.0,250.83,?,?,1.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,NaN,Discharged/transferred to home with home healt...
2,8222157.0,Caucasian,Female,[0-10),?,6,25.0,1.0,1.0,?,Pediatrics-Endocrinology,41.0,0.0,1.0,0.0,0.0,0.0,250.83,?,?,1.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,NaN,Transfer from another health care facility
3,8222157.0,Caucasian,Female,[0-10),?,6,25.0,1.0,1.0,?,Pediatrics-Endocrinology,41.0,0.0,1.0,0.0,0.0,0.0,250.83,?,?,1.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,Discharged/transferred to home with home healt...,NaN
4,8222157.0,Caucasian,Female,[0-10),?,6,25.0,1.0,1.0,?,Pediatrics-Endocrinology,41.0,0.0,1.0,0.0,0.0,0.0,250.83,?,?,1.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,Discharged/transferred to home with home healt...,Discharged/transferred to home with home healt...


In [21]:
df.isna().sum()

patient_nbr                    79
race                           79
gender                         79
age                            79
weight                         79
admission_type_id               4
discharge_disposition_id       79
admission_source_id            79
time_in_hospital               79
payer_code                     79
medical_specialty              79
num_lab_procedures             79
num_procedures                 79
num_medications                79
number_outpatient              79
number_emergency               79
number_inpatient               79
diag_1                         79
diag_2                         79
diag_3                         79
number_diagnoses               79
max_glu_serum                  79
A1Cresult                      79
metformin                      79
repaglinide                    79
nateglinide                    79
chlorpropamide                 79
glimepiride                    79
acetohexamide                  79
glipizide     